

<img src="https://i.imgur.com/XSzy00d.png" style="float:right;width:150px">

**Geoinformationssystem und Python**

# Einleitung

## Lernziele

- Sie wissen, was **georeferenzierte Daten** sind und was ein **GIS** ist
- Sie kennen das Datenformat **GeoJSON**
- Sie können mittels `folium` Daten auf einer **Karte visualisieren**
- Sie kennen **`geopandas`** um GeoJSON Dateien weiterzuverarbeiten


# Georeferenzierte Daten

Ein wichtiger Subtyp von Daten sind die **georeferenzierten Daten**. Solche Daten haben einen Bezug zu einem geografischen Ort, der meist über ein Koordinatenpaar (Längen- und Breitengrad) angegeben wird. Viele Daten, wie Temperatur, Niederschalg, Fahrplan etc., die im täglichen Leben verwendet werden haben einen geografischen Bezug. 

Systeme, die mit georeferenzierten Daten arbeiten, werden als **GIS** (Geografische Informationssysteme) bezeichnet. Sie ermöglichen es uns, komplexe räumliche Zusammenhänge zu verstehen und datengetriebene Entscheidungen zu treffen.


## GeoJSON

Um geografische Daten effizient zu verarbeiten, verwenden wir GeoJSON. GeoJSON ist ein offenes Format zur Darstellung von georeferenzierten Daten mit Hilfe der JavaScript Object Notation (JSON). 

> Mehr Informationen zu GeoJSON gibt es auf der [Wikipedia Seite](https://de.wikipedia.org/wiki/GeoJSON).

In Python können wir die Struktur von **GeoJSON** ganz einfach mit Dictionaries abbilden. Das einfachste GeoJSON-Objekt ist ein Punkt auf der Karte. Um dieses Objekt zu initialisieren, werden die Schlüssel `type` und `coordinates` benötigt.

Nachfolgend ein Beispiel eines GeoJSON-Objekts vom Typ **Point** mit den Koordinaten des Hauptgebäudes der Universität Bern:


In [ ]:
point = {
    "coordinates": [
      7.438155593945083, # longitude
      46.95041833701566  # latitude
    ],
    "type": "Point"
}

Hier übergeben wir also ein Dictionary, welches zwei Keys, `coordinates` und `type` enthält. `coordinates` ist wiederum eine Liste welche zwei Floats enthält welche die [**longitude**](https://de.wikipedia.org/wiki/Geographische_Länge) resp. [**latitude**](https://de.wikipedia.org/wiki/Geographische_Breite) des Punktes repräsentieren. 

<div class="gk-exercise">

- Öffne den folgenden [Link](https://geojson.io) in einem neuen Fenster/Tab
- Suche nach **deiner** Adresse und füge ein Marker hinzu
- kopiere das resultierende GeoJSON Dictionary in die folgende Codezelle und speichere es in der variable my_home

<details>
    <img src="https://i.imgur.com/x5zCVzA.gif">
</details>
    
</div>

In [ ]:
# YOUR CODE HERE

# Folium

Während das **GeoJSON-Format** beschreibt, wie unterschiedliche Geometrien mit geografischem Bezug in strukturierter Form gespeichert werden können, stellt es selbst noch keine Karte dar. Um diese Geometrien visuell auf einer Karte anzuzeigen, benötigen wir eine zusätzliche Bibliothek.  

In Python bietet sich dafür die Library **Folium** an. Sie ermöglicht es, GeoJSON-Daten einfach in interaktive Karten zu integrieren und darzustellen.

> `folium` builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the leaflet.js library. Manipulate your data in Python, then visualize it in on a Leaflet map via folium.
> 
> Weitere Infromationen zu `folium` auf der [Projekt Seite](https://python-visualization.github.io/folium/index.html)

Wir werden nun das Modul `folium` kennenlernen, welches uns Methoden und Funktionen, d.h. eine API, bereitstellt um mit einer Karte zu interagieren. 

In [ ]:
import folium
lat = 46.95
lon = 7.438
folium.Map(location=[lat, lon], zoom_start=17.0)

Die Funktion `Map` liefert ein `Map` Objekt zurück. Da es sich um den letzten Ausdruck der Zelle handelt wird diese direkt angezeigt. Wir möchten jedoch mit dem `Map` Objekt weiterarbeiten, deshalb weisen wir das Objekt der Variable `m` zu:

In [ ]:
m = folium.Map(location=[lat, lon], zoom_start=12.0)

Da der letzte Ausdruck nun eine Zuweisung ist, erfolgt keine direkte Ausgabe.  

> Das Rendern der Karte ist ein ressourcenintensiver Prozess. Deshalb arbeiten wir mit dem Objekt `m` und zeigen es erst am Ende explizit an.  

Um uns mit **Folium** vertraut zu machen, fügen wir den zuvor definierten Punkt `my_home` zur Karte hinzu.

In [ ]:
folium.GeoJson(my_home).add_to(m) 
m

Um neben der Position auch zusätzliche Informationen darzustellen, können beim Marker **Tooltips** eingesetzt werden. Diese werden sichtbar, sobald der Mauszeiger über den Marker bewegt wird.  

Die Inhalte der Tooltips lassen sich zudem mit **HTML-Tags** formatieren. So können beispielsweise Name, Temperatur oder Luftfeuchtigkeit übersichtlich und ansprechend angezeigt werden.


In [ ]:
folium.GeoJson(point, 
    tooltip="Grundkurs Programmieren<br />Universität Bern",
    ).add_to(m)
m

Der Ausdruck `m` am Ende der Zelle führt erneut zur Ausgabe der Karte. Da beide Marker demselben Kartenobjekt hinzugefügt wurden, erscheinen sie gemeinsam auf der Karte.

# Suchfunktion

In vielen Fällen sind nur Adressen, aber keine Koordinaten bekannt. Um die passenden Koordinaten zu einer Adresse zu finden, implementieren wir als Nächstes eine eigene Suchfunktion.

Dabei kommen folgende Konzepte zum Einsatz:
- Zugriff auf eine **Web-API**
- Arbeiten mit einem **Dictionary**
- Definieren von **Funktionen**

Um die Aufgabe zu vereinfachen, beschränken wir uns auf die Schweiz. Das [Bundesamt für Landestopografie (swisstopo)](https://www.swisstopo.admin.ch/de/home.html) stellt uns dafür eine passende API zur Verfügung:

> https://api3.geo.admin.ch/services/sdiservices.html#search

Konkret nutzen wir die [Location Search API](https://api3.geo.admin.ch/services/sdiservices.html#id25). Gemäss Dokumentation müssen bei jeder Anfrage bestimmte Parameter übergeben werden.

Dafür definieren wir eine Variable `API`:

In [ ]:
API = "https://api3.geo.admin.ch/rest/services/api/SearchServer?type=locations&origins=address,parcel&searchText=" 

> Wir schreiben `API` gross um zu zeigen, dass es sich hier um eine [Konstante](https://realpython.com/python-constants/#user-defined-constants) handelt. Also eine Variable welche ihren Wert zur Laufzeit unseres Programms nicht ändert. 

<details>
  Alles was bei einer URL nach einem <code>?</code> geschrieben wird, wird als <a href="https://wiki.selfhtml.org/wiki/URL-Parameter">Get-Parameter</a> bezeichnet. In unserem Fall sind das die Parameter:
  <ul>
    <li><code>type</code> mit dem Wert: <code>locations</code></li>
    <li><code>origins</code> mit dem Wert: <code>address,parcel</code></li>
    <li><code>searchText</code></li>
  </ul>
  die einzelnen Parameter werden mittels <code>&</code> separiert. 
</details>

Um schliesslich unseren Request `url` zu erhalten fügen wir eine entsprechende Adresse hinzu:

In [ ]:
url = API + "Hochschulstrasse 4"

> d.h. wir setzten den Wert für den `searchText` Get-Parameter, siehe Details.

und können dann wie gewohnt mittels dem `requests` Modul eine Anfrage starten:

In [ ]:
import requests
response = requests.get(url).json()

gemäss der [Dokumentation](https://api3.geo.admin.ch/services/sdiservices.html#id26) erhalten wir ein JSON Objekt zurück. Deshalb können wir direkt mit der `json` Methode die Antwort zu einem JSON konvertieren

In [ ]:
response

Die Antwort der API besitzt genau die Struktur, wie sie in der Dokumentation beschrieben wird. Allerdings liegen die Daten nicht im **GeoJSON-Format** vor und müssen daher noch in das entsprechende Format umgewandelt werden.

<div class="gk-exercise"> 

Orte/Punkte auf einer Landkarte werden mit zwei Koordinaten angegeben. Im JSON, das wir von unserem Request zurückerhalten, sind diese ziemlich verschachtelt angegeben. Diese Koordinaten (`lon` und `lat`) sollen aus dem JSON extrahiert werden, um anschliessend an unsere Funktion `GeoJSON` übergeben werden zu können.

Schreibe den Code um auf die entsprechenden Werte zuzugreifen:

<details>
    <summary>Tipps</summary>
    Versuche Schrittweise vorzugehen. <code>lon</code> und <code>lat</code> sind Schlüssel des Dictionary <code>attrs</code>, was wiederum ...
    <details>
    ... Element jedes Listenelements des <code>results</code> Schlüssel ist.
    </details>
</details>

</div>

In [ ]:
# YOUR CODE HERE

m = folium.Map(location=[lat, lon], zoom_start=12.0)
folium.GeoJson(
  {
    "type": "FeatureCollection",
    "features": [
      {
        "type": "Feature",
        "properties": {},
        "geometry": {
          "coordinates": [
            lon,
            lat
          ],
          "type": "Point"
        }
      }
    ]
  }).add_to(m)
m

<div class="gk-exercise"> 

Nun können wir das zu einer Funktion zusammenfassen:

<div>

In [ ]:
def location_search(search_text):
    # replace ist notwenig, untersuche wie sich die Resultate verändern
    # wenn auf replace verzichtet wird. 
    url = API + search_text.replace(" ", ",")
    response = requests.get(url).json()
    
    # YOUR CODE HERE

    return lat, lon

<div class="gk-exercise">

Verwende die Funktion `input`, um eine **Adresssuche** zu implementieren und die Suchergebnisse auf einer Karte darzustellen. Im Tooltip des Markers soll der **Suchbegriff** angezeigt werden.
    
</div>

In [ ]:
# YOUR CODE HERE

# Hitzeinsel-Monitoring

<img src="https://i.imgur.com/RiMPXp5.png" style="float:right;width:150px;margin:20px;">

Bis jetzt haben wir in dieser Lektion gelernt, wie Punkte auf einer Karte dargestellt werden können und wie nach Koordinaten gesucht wird. Nun betrachten wir anhand eines konkreten Beispiels, wie Daten mit geografischem Bezug verarbeitet werden können.

Temperaturmessungen gehören zu den am häufigsten verwendeten Geodaten. In dieser und der nächsten Lektion nutzen wir Temperatur- und Luftfeuchtigkeitsdaten aus einem **Hitzeinsel-Monitoring**, das von der **Stadtklimagruppe des Geographischen Instituts der Universität Bern (GIUB)** betrieben wird.

Mehr Informationen: [Stadtklima Bern – GIUB](https://www.geography.unibe.ch/forschung/gruppe_fuer_klimatologie/forschungsprojekte/stadtklima_bern/index_ger.html)

Die Sensordaten werden über die [Smart Urban Heat Map](https://smart-urban-heat-map.ch) als **Open Data API** zur Verfügung gestellt. 

Die API bietet zwei Endpoints:

- **Latest Data**: `https://smart-urban-heat-map.ch/api/v2/latest`
Liefert eine Liste aller Sensoren mit den aktuellsten Messwerten (Lektion 11).

- **Time Series**: `https://smart-urban-heat-map.ch/api/v2/timeseries?stationId=STATION_ID&date_from=...&date_to=...` 
Liefert für einen spezifischen Sensor die Messwerte in einem definierten Zeitraum (Lektion 12).




Im ersten Schritt untersuchen wir, welche Daten der *Latest*-Endpunkt liefert. Dazu rufen wir den Endpunkt mittels `requests.get(...)` auf und geben das Resultat mit `display` aus.


In [ ]:
import requests
import json

response = requests.get("https://smart-urban-heat-map.ch/api/v2/latest")
data = response.json()

display(data)


Das Resultat ist eine **FeatureCollection** im **GeoJSON-Format**. Das JSON enthält die Liste `features`. Jedes `feature` repräsentiert einen Sensor des Messnetzes mit den zuletzt gemessenen Messwerten. 

Neu finden wir in diesen Daten den Schlüssel `properties`. Im GeoJSON-Format sind **properties** Eigenschaften, die beliebig erweitert werden können. Jede Eigenschaft besteht aus einem Namen und dem zugehörigen Wert.


<div class="gk-exercise">

Selektiere den **ersten Sensor** und gib mit `print` die **Eigenschaften** sowie die **Koordinaten** formatiert aus. 
 
</div>

In [ ]:
# YOUR CODE 

Da die Daten bereits im **GeoJSON-Format** zurückgegeben werden, können wir die Standorte der Sensoren direkt auf der Karte einblenden.

In [ ]:
import folium

m = folium.Map([46.95, 7.438], zoom_start=12)
folium.GeoJson(data).add_to(m)

m

<div class="gk-exercise">

Nur mit den Punkten ist die Karte noch wenig informativ. Erweitere die Karte, indem du im Tooltip der Standorte folgende Informationen anzeigst:

- `name`
- `dateObserved`
- `temperature`
- `relativeHumidity`

<details>
  <summary>Tipps</summary>
  Erstelle eine Schleife («for»-Loop) über alle Features in <b>data['features']</b> und füge die Marker einzeln der Karte hinzu.
</details>


</div>

In [ ]:
# YOUR CODE HERE

# GeoPandas

In der [Lektion 10](Lektion_10/NumPy_Pandas.ipynb#Pandas) haben wir bereits **Pandas** kennengelernt. In diesem Abschnitt lernen wir **GeoPandas** kennen, eine Erweiterung von Pandas für **GeoJSON**-Daten.  

> GeoPandas ist ein Open-Source-Projekt, das die Arbeit mit geospatialen Daten in Python vereinfacht.  
> GeoPandas erweitert die Datentypen von Pandas, sodass auch **räumliche Operationen** auf geometrischen Typen möglich sind.  
> Weitere Informationen zu `GeoPandas` findest du auf der [Projektseite](https://geopandas.org/en/stable/).

GeoPandas erlaubt es uns also, unser Wissen aus Pandas direkt auf **GeoJSON-Dateien** anzuwenden.  
Als Erstes erstellen wir ein **GeoDataFrame**. Dies können wir direkt aus den `features` des *Latest*-Endpoints machen.


In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame.from_features(data["features"])

Wie gewohnt können wir uns mittels der Methode head einen ersten Überblick verschaffen.

In [ ]:
gdf.head()

<div class="gk-exercise">

Unter den Spalten befinden sich die Spalten `outdated` und `measurementsPlausible`. Die Spalte `outdated` gibt als Boolean an, ob der Messwert **veraltet** ist, und `measurementsPlausible` zeigt an, ob der Messwert **plausibel** ist.

Verwende diese beiden Eigenschaften, um alle **aktuellen** und **plausiblen** Daten in einem neuen Dataframe `recent_data` zu speichern.

</div>

In [ ]:
# YOUR CODE HERE

# Schlussaufgabe

<div class="gk-exercise">

Analysiere die gefilterten Daten (`recent_data`) und beantworte die folgenden Fragen mittels formatierten `print`-Statements:

1. Wie viele Messstationen gibt es insgesamt?  
2. Welche ist die **höchste**, **niedrigste** und **mittlere** Temperatur?  
3. Welche ist die **höchste**, **niedrigste** und **mittlere** Luftfeuchtigkeit?  
4. Selektiere den Sensor mit der höchsten Temperatur und gib diesen mittels **Folium** auf einer Karte aus.

<details>
    <summary>Tipps</summary>
    Verwende <b>df["spalte"].max()</b> um den maximalen Wert einer Spalte zu ermitteln.  
    <details>
        <summary>Tipp</summary>
        Filtere alle Sensoren, die die maximale Temperatur haben, um den Sensor mit der höchsten Temperatur zu selektieren. 
    </details>
    <details>
        <summary>Tipp</summary>
        Verwende <b>df.to_json()</b>, um GeoPandas-Daten wieder in ein GeoJSON umzuwandeln und in der Karte einzufügen. 
    </details>
</details>

</div>


In [ ]:
# YOUR CODE HERE

# Zusammenfassung

In diesem Notebook wurde gezeigt, wie **georeferenzierte Daten** mit Koordinatenpaaren geografische Orte beschreiben und wie diese strukturiert im **GeoJSON-Format** gespeichert werden können.  

Die Bibliothek **Folium** wurde vorgestellt, um interaktive Karten zu erstellen und GeoJSON-Daten zu visualisieren. Ausserdem wurde gezeigt, wie Adressen mithilfe der **Swisstopo API** in Koordinaten umgewandelt werden können.  

Schliesslich wurde **GeoPandas** eingeführt, welches Pandas um geografische Funktionen erweitert und die Analyse räumlicher Daten ermöglicht.  
Am praktischen Beispiel des **Hitzeinsel-Monitorings** wurde demonstriert, wie echte Sensordaten von einer API abgerufen, verarbeitet und auf einer interaktiven Karte dargestellt werden können.



# Impressum

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg" /></a><br />Dieses Werk ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Namensnennung - Weitergabe unter gleichen Bedingungen 4.0 International Lizenz</a>.

Autoren: [Jakob Schärer](mailto:jakob.schaerer@unibe.ch), [Lionel Stürmer](mailto:lionel.stuermer@bfh.ch) <br>
Ursprünglicher Text von: Noe Thalheim, Benedikt Hitz-Gamper


## Credits

- https://rsandstroem.github.io/tag/folium.html
- https://python-visualization.github.io/folium/quickstart.html
- https://smart-urban-heat-map.ch


```
Why do programmers prefer dark mode?
Because light attracts bugs.
```